# 02. Exploratory Data Analysis (EDA)

## 1. Objective
- Phân tích hành vi tiêu thụ điện
- Phát hiện các mô hình thời vụ

## 2. Time Series Visualization
- Chuỗi thời gian tổng
- Chuỗi theo consumer (mẫu đại diện)

## 3. Seasonality Analysis
### 3.1 Daily Seasonality
- Biểu đồ trung bình theo giờ

### 3.2 Weekly Seasonality
- Ngày trong tuần vs cuối tuần

### 3.3 Yearly Seasonality
- Biến động theo tháng / mùa

## 4. Load Distribution
- Histogram
- Boxplot

## 5. Statistical Summary
- Mean, Median, Std
- Min / Max

## 6. Key Insights
- Các mẫu thời vụ quan trọng
- Nhận dạng giờ cao điểm / thấp điểm

## 7. Conclusion
- Tóm tắt insight chính phục vụ modeling